In [40]:
from collections import defaultdict, Counter

import graphviz
import yaml
import json
import os

In [38]:
coverage_path = 'data/coverage_14.yaml'
func_graph_path = 'data/main.dot'
mapping_path = 'data/mapping.json'

In [3]:
mapping = json.load(open(mapping_path, 'r'))

In [4]:
coverage = yaml.load(open(coverage_path, 'r'))

In [11]:
groups = list(coverage.keys())
print(groups)

[14, 17, 27, 46, 89, 112, 139, 154]


In [21]:
covered_set = set()

In [24]:
for group, runs in coverage.items():
    for run, blocks in runs.items():
        covered_set |= set(blocks)

In [25]:
print(len(covered_set))

4056


In [27]:
overlaps = Counter()

In [33]:
for function, blocklist in mapping.items():
    overlaps[function] = len(covered_set & set([int(b, 16) for b in blocklist]))

In [39]:
overlaps.most_common(10)

[('start_pass', 158),
 ('_unknown', 124),
 ('decode_mcu', 115),
 ('decode_mcu_AC_refine', 94),
 ('decompress_smooth_data', 94),
 ('jpeg_make_d_derived_tbl', 92),
 ('get_dht', 85),
 ('get_sof', 78),
 ('get_sos', 76),
 ('start_pass_huff_decoder', 71)]

In [35]:
sum(overlaps.values())

3039

In [44]:
address_label = {}

In [49]:
with open(func_graph_path, 'r') as in_file:
    with open('res/main_compressed.dot', 'w') as out_file:
        
        for line in in_file.readlines():
            
            if 'label=' in line:
                address = line.split()[0]
                content = line.split('label=')[1].split(']')[0]
                address_label[address] = content
                new_line = line.split('label=')[0] + 'label=' + address + ']' + os.linesep
                out_file.write(new_line)
            
            else:
                out_file.write(line)

In [51]:
json.dump(address_label, open('res/main_label_content_mapping.json', 'w'), indent=2)